In [10]:
import xlwings as xw
import pandas as pd

from multiprocessing.dummy import Pool as ThreadPool
import bs4 as bs
import requests 
import re
import yahoo_fin.stock_info as yf
import yfinance as yf1
import pandas_datareader.data as web
import datetime
from dateutil.relativedelta import relativedelta

list_wrong = []

result_PE =pd.DataFrame()

i = 0
    
def findata(stock):
    global result_PE
    global i 
    i = i + 1
    try:
        #print(stock)
        #爬虫爬取数据
        stock1 = stock
        stock = stock.replace("-", ".")
        print(i,stock)
        url = "https://www.macrotrends.net/stocks/charts/"+stock+"//eps-earnings-per-share-diluted"
        resp = requests.get(url)
        soup = bs.BeautifulSoup(resp.text, "lxml")
        table = soup.findAll('tbody')
        tickers = []
        tickers1 = []
        
        
        for row in table[2].findAll('tr')[0:]:
            ticker = [t.text.strip('$') for t in row.find_all('td')] #去掉$符号
            tickers1.append(ticker)
        #print(tickers1)

        for row in table[1].findAll('tr')[0:]:
            ticker = [t.text.strip('$') for t in row.find_all('td')] #去掉$符号
            tickers.append(ticker)

        #print(tickers)
        #计算 EPS_TTM
        columns=["Date", "EPS"]
        df_EPS = pd.DataFrame(data=tickers, columns=columns)
        df_EPS['EPS'] = df_EPS['EPS'].astype(float) 
        df_EPS['EPS_TTM'] = df_EPS['EPS'] + df_EPS['EPS'].shift(-1) + df_EPS['EPS'].shift(-2) + df_EPS['EPS'].shift(-3) 
        df_EPS.EPS_TTM = df_EPS.EPS_TTM.fillna(-998) #-998表示没有可供计算年EPS的数据
        
        #根据其它列的条件，增加新列EPS_GRTH
        df_EPS.loc[((df_EPS['EPS_TTM'].shift(-4)>0) & (df_EPS['EPS_TTM']>0)),'EPS_TTM_GRTH']=df_EPS['EPS_TTM']/df_EPS['EPS_TTM'].shift(-4) - 1 
        df_EPS.loc[((df_EPS['EPS'].shift(-4)>0) & (df_EPS['EPS']>0)),'EPS_GRTH']=df_EPS['EPS']/df_EPS['EPS'].shift(-4) - 1 
        
        #df_EPS.EPS_TTM_GRTH = df_EPS.EPS_TTM_GRTH.fillna(-999)
        #df_EPS.EPS_GRTH = df_EPS.EPS_GRTH.fillna(-999)
        
        #print(df_EPS)
        #增加收入信息
        url2 = "https://www.macrotrends.net/stocks/charts/"+stock+"//revenue"
        resp2 = requests.get(url2)
        soup2 = bs.BeautifulSoup(resp2.text, "lxml")
        table = soup2.findAll('tbody')
        tickers2 = []
        
       # print(table[1])
        for row in table[1].findAll('tr')[0:]:
            
            ticker = [t.text.strip('$').replace(',','') for t in row.find_all('td')] #去掉$符号
            #print(ticker)
            if ticker[1] !='':
                #print(ticker[1])
                tickers2.append(ticker)
        #print(tickers2)
             
        
        columns2=["Date", "Revenue"]
        df_Revenue = pd.DataFrame(data=tickers2, columns=columns2)
        #print(df_Revenue['Revenue'])
        df_Revenue['Revenue'] = df_Revenue['Revenue'].astype(float) 
        #print(2)
        df_Revenue['Revenue_TTM'] = df_Revenue['Revenue'] + df_Revenue['Revenue'].shift(-1) + df_Revenue['Revenue'].shift(-2) + df_Revenue['Revenue'].shift(-3) 
        #print(3)
        df_Revenue.Revenue_TTM = df_Revenue.Revenue_TTM.fillna(-998) #-998表示没有可供计算年EPS的数据
        
        
        
        #根据其它列的条件，增加新列EPS_GRTH
        df_Revenue.loc[((df_Revenue['Revenue_TTM'].shift(-4)>0) & (df_Revenue['Revenue_TTM']>0)),'Revenue_TTM_GRTH']=df_Revenue['Revenue_TTM']/df_Revenue['Revenue_TTM'].shift(-4) - 1 
        df_Revenue.loc[((df_Revenue['Revenue'].shift(-4)>0) & (df_Revenue['Revenue']>0)),'Revenue_GRTH']=df_Revenue['Revenue']/df_Revenue['Revenue'].shift(-4) - 1 
        
        df_Revenue.Revenue_TTM_GRTH = df_Revenue.Revenue_TTM_GRTH.fillna(-999)
        df_Revenue.Revenue_GRTH = df_Revenue.Revenue_GRTH.fillna(-999)
        
        end = datetime.date.today()
        start = end - relativedelta(days=10)
              
        Price = web.get_data_yahoo(stock1,start,end).iloc[-1]['Adj Close']
        
        #Date = web.get_data_yahoo(stock1,start,end).iloc[-1].idx
        
        if df_EPS.iloc[0]['EPS_TTM'] > 0:
            PE = Price/df_EPS.iloc[0]['EPS_TTM']
        else:
            PE = 0
        
        if df_EPS.iloc[0]['EPS_TTM_GRTH'] > 0:
            PEG = PE*0.01/df_EPS.iloc[0]['EPS_TTM_GRTH']
        else:
            PEG = 0
                 
        result_PE = result_PE.append(pd.DataFrame({'ticker':[stock],\
                                    
                                    'PE':[PE],
                                    'PEG':[PEG],
                                    'Price':[Price],
                                    #'Date':[Date],
                                    'MarketCap':[tickers1[0][2].strip('B')],
                                    'Industry': [tickers1[0][1]],                                                                            
 
                                                          
                                    'Q12':[df_EPS.iloc[11]['Date']],
                                    'Q11':[df_EPS.iloc[10]['Date']],
                                    'Q10':[df_EPS.iloc[9]['Date']],
                                    'Q9':[df_EPS.iloc[8]['Date']],
                                    'Q8':[df_EPS.iloc[7]['Date']],
                                    'Q7':[df_EPS.iloc[6]['Date']],
                                    'Q6':[df_EPS.iloc[5]['Date']],
                                    'Q5':[df_EPS.iloc[4]['Date']],
                                    'Q4':[df_EPS.iloc[3]['Date']],
                                    'Q3':[df_EPS.iloc[2]['Date']],
                                    'Q2':[df_EPS.iloc[1]['Date']],
                                    'Q1':[df_EPS.iloc[0]['Date']],      
                                                   
                                    'EPS12':[df_EPS.iloc[11]['EPS']],
                                    'EPS11':[df_EPS.iloc[10]['EPS']],                                              
                                    'EPS10':[df_EPS.iloc[9]['EPS']],
                                    'EPS9':[df_EPS.iloc[8]['EPS']],
                                    'EPS8':[df_EPS.iloc[7]['EPS']],
                                    'EPS7':[df_EPS.iloc[6]['EPS']],                                              
                                    'EPS6':[df_EPS.iloc[5]['EPS']],
                                    'EPS5':[df_EPS.iloc[4]['EPS']],
                                    'EPS4':[df_EPS.iloc[3]['EPS']],
                                    'EPS3':[df_EPS.iloc[2]['EPS']],                                              
                                    'EPS2':[df_EPS.iloc[1]['EPS']],
                                    'EPS1':[df_EPS.iloc[0]['EPS']],
                                                   
                                    'EPS_GRTH12':[df_EPS.iloc[11]['EPS_GRTH']],
                                    'EPS_GRTH11':[df_EPS.iloc[10]['EPS_GRTH']],                                              
                                    'EPS_GRTH10':[df_EPS.iloc[9]['EPS_GRTH']],
                                    'EPS_GRTH9':[df_EPS.iloc[8]['EPS_GRTH']], 
                                    'EPS_GRTH8':[df_EPS.iloc[7]['EPS_GRTH']],
                                    'EPS_GRTH7':[df_EPS.iloc[6]['EPS_GRTH']],                                              
                                    'EPS_GRTH6':[df_EPS.iloc[5]['EPS_GRTH']],
                                    'EPS_GRTH5':[df_EPS.iloc[4]['EPS_GRTH']], 
                                    'EPS_GRTH4':[df_EPS.iloc[3]['EPS_GRTH']],
                                    'EPS_GRTH3':[df_EPS.iloc[2]['EPS_GRTH']],                                              
                                    'EPS_GRTH2':[df_EPS.iloc[1]['EPS_GRTH']],
                                    'EPS_GRTH1':[df_EPS.iloc[0]['EPS_GRTH']], 
                                                   
                                    'EPS_TTM12':[df_EPS.iloc[11]['EPS_TTM']],
                                    'EPS_TTM11':[df_EPS.iloc[10]['EPS_TTM']],                                              
                                    'EPS_TTM10':[df_EPS.iloc[9]['EPS_TTM']],
                                    'EPS_TTM9':[df_EPS.iloc[8]['EPS_TTM']],  
                                    'EPS_TTM8':[df_EPS.iloc[7]['EPS_TTM']],
                                    'EPS_TTM7':[df_EPS.iloc[6]['EPS_TTM']],                                              
                                    'EPS_TTM6':[df_EPS.iloc[5]['EPS_TTM']],
                                    'EPS_TTM5':[df_EPS.iloc[4]['EPS_TTM']],  
                                    'EPS_TTM4':[df_EPS.iloc[3]['EPS_TTM']],
                                    'EPS_TTM3':[df_EPS.iloc[2]['EPS_TTM']],                                              
                                    'EPS_TTM2':[df_EPS.iloc[1]['EPS_TTM']],
                                    'EPS_TTM1':[df_EPS.iloc[0]['EPS_TTM']],  
                                                   
                                    'EPS_TTM_GRTH12':[df_EPS.iloc[11]['EPS_TTM_GRTH']],
                                    'EPS_TTM_GRTH11':[df_EPS.iloc[10]['EPS_TTM_GRTH']],                                              
                                    'EPS_TTM_GRTH10':[df_EPS.iloc[9]['EPS_TTM_GRTH']],
                                    'EPS_TTM_GRTH9':[df_EPS.iloc[8]['EPS_TTM_GRTH']],  
                                    'EPS_TTM_GRTH8':[df_EPS.iloc[7]['EPS_TTM_GRTH']],
                                    'EPS_TTM_GRTH7':[df_EPS.iloc[6]['EPS_TTM_GRTH']],                                              
                                    'EPS_TTM_GRTH6':[df_EPS.iloc[5]['EPS_TTM_GRTH']],
                                    'EPS_TTM_GRTH5':[df_EPS.iloc[4]['EPS_TTM_GRTH']],
                                    'EPS_TTM_GRTH4':[df_EPS.iloc[3]['EPS_TTM_GRTH']],
                                    'EPS_TTM_GRTH3':[df_EPS.iloc[2]['EPS_TTM_GRTH']],                                              
                                    'EPS_TTM_GRTH2':[df_EPS.iloc[1]['EPS_TTM_GRTH']],
                                    'EPS_TTM_GRTH1':[df_EPS.iloc[0]['EPS_TTM_GRTH']],
                                                   
                                    'Revenue12':[df_Revenue.iloc[11]['Revenue']],
                                    'Revenue11':[df_Revenue.iloc[10]['Revenue']],                                              
                                    'Revenue10':[df_Revenue.iloc[9]['Revenue']],
                                    'Revenue9':[df_Revenue.iloc[8]['Revenue']],
                                    'Revenue8':[df_Revenue.iloc[7]['Revenue']],
                                    'Revenue7':[df_Revenue.iloc[6]['Revenue']],                                              
                                    'Revenue6':[df_Revenue.iloc[5]['Revenue']],
                                    'Revenue5':[df_Revenue.iloc[4]['Revenue']],
                                    'Revenue4':[df_Revenue.iloc[3]['Revenue']],
                                    'Revenue3':[df_Revenue.iloc[2]['Revenue']],                                              
                                    'Revenue2':[df_Revenue.iloc[1]['Revenue']],
                                    'Revenue1':[df_Revenue.iloc[0]['Revenue']],
                                                   
                                    'Revenue_GRTH12':[df_Revenue.iloc[11]['Revenue_GRTH']],
                                    'Revenue_GRTH11':[df_Revenue.iloc[10]['Revenue_GRTH']],                                         
                                    'Revenue_GRTH10':[df_Revenue.iloc[9]['Revenue_GRTH']],
                                    'Revenue_GRTH9':[df_Revenue.iloc[8]['Revenue_GRTH']],             
                                    'Revenue_GRTH8':[df_Revenue.iloc[7]['Revenue_GRTH']],
                                    'Revenue_GRTH7':[df_Revenue.iloc[6]['Revenue_GRTH']],                                              
                                    'Revenue_GRTH6':[df_Revenue.iloc[5]['Revenue_GRTH']],
                                    'Revenue_GRTH5':[df_Revenue.iloc[4]['Revenue_GRTH']],              
                                    'Revenue_GRTH4':[df_Revenue.iloc[3]['Revenue_GRTH']],
                                    'Revenue_GRTH3':[df_Revenue.iloc[2]['Revenue_GRTH']],                                              
                                    'Revenue_GRTH2':[df_Revenue.iloc[1]['Revenue_GRTH']],
                                    'Revenue_GRTH1':[df_Revenue.iloc[0]['Revenue_GRTH']],                             
                                               
                                                  
                                    'Revenue_TTM12':[df_Revenue.iloc[11]['Revenue_TTM']],
                                    'Revenue_TTM11':[df_Revenue.iloc[10]['Revenue_TTM']],                                              
                                    'Revenue_TTM10':[df_Revenue.iloc[9]['Revenue_TTM']],
                                    'Revenue_TTM9':[df_Revenue.iloc[8]['Revenue_TTM']],  
                                    'Revenue_TTM8':[df_Revenue.iloc[7]['Revenue_TTM']],
                                    'Revenue_TTM7':[df_Revenue.iloc[6]['Revenue_TTM']],                                              
                                    'Revenue_TTM6':[df_Revenue.iloc[5]['Revenue_TTM']],
                                    'Revenue_TTM5':[df_Revenue.iloc[4]['Revenue_TTM']],  
                                    'Revenue_TTM4':[df_Revenue.iloc[3]['Revenue_TTM']],
                                    'Revenue_TTM3':[df_Revenue.iloc[2]['Revenue_TTM']],                                              
                                    'Revenue_TTM2':[df_Revenue.iloc[1]['Revenue_TTM']],
                                    'Revenue_TTM1':[df_Revenue.iloc[0]['Revenue_TTM']],  
                                                   
                                    'Revenue_TTM_GRTH12':[df_Revenue.iloc[11]['Revenue_TTM_GRTH']],
                                    'Revenue_TTM_GRTH11':[df_Revenue.iloc[10]['Revenue_TTM_GRTH']],                                              
                                    'Revenue_TTM_GRTH10':[df_Revenue.iloc[9]['Revenue_TTM_GRTH']],
                                    'Revenue_TTM_GRTH9':[df_Revenue.iloc[8]['Revenue_TTM_GRTH']], 
                                    'Revenue_TTM_GRTH8':[df_Revenue.iloc[7]['Revenue_TTM_GRTH']],
                                    'Revenue_TTM_GRTH7':[df_Revenue.iloc[6]['Revenue_TTM_GRTH']],                                              
                                    'Revenue_TTM_GRTH6':[df_Revenue.iloc[5]['Revenue_TTM_GRTH']],
                                    'Revenue_TTM_GRTH5':[df_Revenue.iloc[4]['Revenue_TTM_GRTH']], 
                                    'Revenue_TTM_GRTH4':[df_Revenue.iloc[3]['Revenue_TTM_GRTH']],
                                    'Revenue_TTM_GRTH3':[df_Revenue.iloc[2]['Revenue_TTM_GRTH']],                                              
                                    'Revenue_TTM_GRTH2':[df_Revenue.iloc[1]['Revenue_TTM_GRTH']],
                                    'Revenue_TTM_GRTH1':[df_Revenue.iloc[0]['Revenue_TTM_GRTH']], 
                                                 
                                                                       
                                    'Intro':[tickers1[1][0][1:-1]]
                                         
                                     }),ignore_index=True,sort=False) 
        #print(tickers1[1])
       
    except:
        print(stock,'.....................wrong.....................')
        list_wrong.append(stock)
        
df_p_e_f2 =pd.DataFrame()

list_wrong1 = []
def trace_pe(stock):
    
    global df_p_e_f2
    try:
        #print(stock)
        #爬虫爬取数据
        url = "https://www.macrotrends.net/stocks/charts/"+stock+"//eps-earnings-per-share-diluted"
        resp = requests.get(url)
        soup = bs.BeautifulSoup(resp.text, "lxml")
        table = soup.findAll('tbody')
        tickers = []

        for row in table[1].findAll('tr')[0:]:
            ticker = [t.text.strip('$') for t in row.find_all('td')] #去掉$符号
            tickers.append(ticker)

        #print(tickers)
        #计算 EPS_TTM
        columns=["Date", "EPS"]
        df_EPS = pd.DataFrame(data=tickers, columns=columns)
        df_EPS['EPS'] = df_EPS['EPS'].astype(float) 
        df_EPS['EPS_TTM'] = df_EPS['EPS'] + df_EPS['EPS'].shift(-1) + df_EPS['EPS'].shift(-2) + df_EPS['EPS'].shift(-3) 
        df_EPS.EPS_TTM = df_EPS.EPS_TTM.fillna(-998) #-998表示没有可供计算年EPS的数据
        
        #根据其它列的条件，增加新列EPS_GRTH
        df_EPS.loc[((df_EPS['EPS_TTM'].shift(-4)>0) & (df_EPS['EPS_TTM']>0)),'EPS_GRTH']=df_EPS['EPS_TTM']/df_EPS['EPS_TTM'].shift(-4) - 1 
        df_EPS.EPS_GRTH = df_EPS.EPS_GRTH.fillna(-999)
        
        
        df_EPS['Date'] = pd.to_datetime(df_EPS['Date'])
        df_EPS.set_index('Date', inplace=True) 
        
        #print(df_EPS)
        #下载股价数据
        #start = datetime.datetime(2005,4,1)
        end = datetime.date.today()
        start = end - relativedelta(years=3)
        df = web.get_data_yahoo(stock,start,end)
        df1 = df[['Volume','Adj Close']]
        
        #print(df1.head(10))
        #拼表#填充
        df_p_e = pd.concat([df1,df_EPS],axis=1)            
        df_p_e_f = df_p_e.fillna(method='pad')
        
        #print(df_p_e_f.head(10))
        
        #计算PE、PEG、分位数、均值
        df_p_e_f.loc[df_p_e_f['EPS_TTM']>0,'PE']=df_p_e_f['Adj Close']/df_p_e_f['EPS_TTM']
        
        df_p_e_f['PE_Median_500'] = df_p_e_f.PE.rolling(500).quantile(.55)
        df_p_e_f['PE_quant10_500'] = df_p_e_f.PE.rolling(500).quantile(.1)
        df_p_e_f['PE_quant90_500'] = df_p_e_f.PE.rolling(500).quantile(0.9)
        df_p_e_f['PE_mean_500'] = df_p_e_f.PE.rolling(500).mean()
        
        df_p_e_f.loc[df_p_e_f['EPS_GRTH']>0,'PEG']=df_p_e_f['PE']*0.01/df_p_e_f['EPS_GRTH']
        
        #print(df_p_e_f.head(10))
        #倒序
        df_p_e_f1 = df_p_e_f.sort_index(ascending=False,axis=0)
        
        #print(df_p_e_f1.head(10))
        
        #切片最近10年数据
        df_p_e_f2 = df_p_e_f1.iloc[:250].sort_index(ascending=True,axis=0)
        
        wb = xw.Book('C:/Users/zty1979/Documents/美股/dashboard.xlsx')
        
        sht = wb.sheets[stock]
            
        sht.range('Z1').value = df_p_e_f2
 
    except:
        print(stock,'.....................wrong.....................')
        list_wrong1.append(stock)
        
wb = xw.Book('C:/Users/zty1979/Documents/美股/dashboard.xlsx')
sht_list = wb.sheets['list']
sht_data = wb.sheets['data']
sht_template = wb.sheets['template']

list_tickers = sht_list.range('A2').expand().value

In [11]:
#提取现有sheet名称中的股票名称
x = []# 创建空列表
num =len(wb.sheets)#获取sheet个数
for j in range(0,num):
    sht = wb.sheets[j]
    x.append(sht.name)
    j +=1          # 计数数量自加1

In [12]:
# 删除多余的sheet
for ticker in x:
     if (ticker not in list_tickers) and (ticker not in ['list','data','template'])
         wb.sheets[ticker].delete()
            #sht.delete()

In [13]:
# 增加列表中的sheet
for ticker in list_tickers:
    if ticker not in x:
        sht_template.api.Copy(Before=sht_template.api)
        wb.sheets['template (2)'].api.Name = ticker

In [14]:
# 导入财务数据
tickers = list_tickers 
result_PE=result_PE.drop(index=result_PE.index)
pool = ThreadPool()
pool.map(findata, tickers)
pool.close()
pool.join()

123  TMO
 4 GOOGLALB

NVDA
5 MSFT
6 NUE
7 WIRE
8 LRCX
9 QRVO
10 STLD
11 TSLA
12 AMD
13 TSM
14 PYPL
15 MOS
16 NFLX


In [15]:
list_wrong

[]

In [16]:
tickers = list_wrong
#tickers = ['AAPL','AMD','BA','V']
#tickers = ['JPM']
list_wrong = []
#result_PE=result_PE.drop(index=result_PE.index)
pool = ThreadPool()
pool.map(findata, tickers)
pool.close()
pool.join()

In [17]:
sht_data.range('A1').value = result_PE

In [18]:
for ticker in list_tickers:
    trace_pe(ticker)